In [ ]:
import pandas as pd
import geopandas as gpd
import datetime as date

import matplotlib.pyplot as plt
from shapely.geometry import Point

Reading the "Traffic_Crashes_-_People.csv" file into the variable crashes to create a pandas dataframe.

In [ ]:
crashes = pd.read_csv('Traffic_Crashes_-_People.csv', low_memory = False)
crashes2 = pd.read_csv('Traffic_Crashes_-_Vehicles.csv', low_memory = False)
# print(crashes.columns.values.tolist())
# print(crashes2.columns.values.tolist())

In [ ]:
crashes2F = crashes2.filter(['CRASH_RECORD_ID', 'MAKE', 'MODEL', 'VEHICLE_TYPE'])

Converting the data in the 'CRASH_DATE' column from date and time type to just date type using the datetime library

In [ ]:
crashes['CRASH_DATE'] = pd.to_datetime(crashes['CRASH_DATE']).dt.date

Extracting only the data from 8/1/2020 to 8/1/2022 and stored inside the variable dates.

In [ ]:
dBefore = date.date(2020, 1, 1)
dAfter = date.date(2022, 1, 1)
dates = crashes[(crashes['CRASH_DATE'] > dBefore) & (crashes['CRASH_DATE'] < dAfter)]

Dropping the columns that will not be used in the future cells to visualize data.

In [ ]:
toDrop = ['RD_NO', 'VEHICLE_ID', 'SEAT_NO', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'AIRBAG_DEPLOYED', 'EJECTION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']
dbCrashes = dates.drop(columns = toDrop)
dbCrashes = dbCrashes.dropna(thresh = 10)
dbCrashes.to_csv('Project2.csv', encoding='utf-8', index = False)
dbCrashes.shape

(255434, 11)

In [ ]:
# FOR THE FIRST VISUALIZARTION
merged = pd.merge(crashes2F, dbCrashes, on = "CRASH_RECORD_ID")
tempMerged = merged
merged = merged.dropna(thresh = 14)
merged = merged.drop(columns = "CRASH_RECORD_ID")

merged = merged[(merged['PERSON_TYPE'] == 'DRIVER')]
merged = merged[(merged['SEX'] != 'X')]
merged = merged[(merged['AGE'] > 0) & (merged['AGE'] < 101)] 
merged["AGE_FREQUENCY"] = 1
merged = merged.groupby(["AGE", "SEX"])["AGE_FREQUENCY"].sum()
# merged = merged.groupby(level=0)
merged = merged.reset_index()
merged.to_csv('sex.csv', index = False)
# merged.head()
merged.shape

(197, 3)

In [ ]:
# FOR THE SECOND VISUALIZARTION

bubble = tempMerged.filter(['MAKE', 'MODEL', 'VEHICLE_TYPE'])
bubble = bubble[(bubble['MODEL'] != "OTHER (EXPLAIN IN NARRATIVE)")]

bubble['VEHICLE'] = bubble['MAKE'] + ' ' + bubble['MODEL']

bubble['VALUE'] = 1

bubble = bubble.groupby(["MODEL", "MAKE", "VEHICLE_TYPE"])["VALUE"].sum()
bubble = bubble.reset_index()

bubble['VALUESTR'] = bubble['VALUE'].astype(str)

bubble['TITLE'] = bubble['MAKE'] + " " + bubble['MODEL'] +' : ' + bubble['VALUESTR']
bubble = bubble[(bubble['VALUE'] > 445)]

vTypes = ["BUS OVER 15 PASS.", "MOTORCYCLE (OVER 150CC)", "PASSENGER", "PICKUP", "SPORT UTILITY VEHICLE (SUV)", "VAN/MINI-VAN", "TRUCK - SINGLE UNIT"]
bubble = bubble[bubble['VEHICLE_TYPE'].isin(vTypes)]

bubble.to_csv('Vehicles.csv', index = False)
# bubble.head()
bubble.shape

(191, 6)

In [ ]:
dateBefore2020 = date.date(2020, 1, 1)
dateAfter2020 = date.date(2021, 1, 1)
dateBefore2021 = date.date(2021, 1, 1)
dateAfter2021 = date.date(2022, 1, 1)

In [ ]:
# FOR THE THIRD VISUALIZARTION

dates2020 = crashes[(crashes['CRASH_DATE'] > dateBefore2020) & (crashes['CRASH_DATE'] < dateAfter2020)]
dates2021 = crashes[(crashes['CRASH_DATE'] > dateBefore2021) & (crashes['CRASH_DATE'] < dateAfter2021)]
dates2020 = dates2020.filter(['CRASH_DATE', 'PERSON_ID'])
dates2021 = dates2021.filter(['CRASH_DATE', 'PERSON_ID'])
dates = crashes[(crashes['CRASH_DATE'] > dateBefore2020) & (crashes['CRASH_DATE'] < dateAfter2021)]
dates = dates.filter(['CRASH_DATE', 'PERSON_ID'])
dates = dates.groupby('CRASH_DATE').count()
dates.rename(columns = {'PERSON_ID': 'close'}, inplace = True)
dates = dates.reset_index()
dates.to_csv('dates.csv', encoding='utf-8', index = False)
dates.head()

,CRASH_DATE,close
0,2020-01-02,120
1,2020-01-03,145
2,2020-01-04,136
3,2020-01-05,83
4,2020-01-06,173
